<a href="https://colab.research.google.com/github/tonchan1216/bitbot/blob/master/notebook/get_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import requests,time
from google.colab import drive
import copy

drive.mount('/content/drive/')
exp_dir = 'Colab Notebooks/bitbot/training_data/'
csv_path1 = '/content/drive/My Drive/' + exp_dir + 'btc_all.csv'
csv_path2 = '/content/drive/My Drive/' + exp_dir + 'btc_train.csv'
csv_path3 = '/content/drive/My Drive/' + exp_dir + 'btc_test.csv'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def get_OHLC(before,after):
    duration = 86400
    url = 'https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc'
    query = {
        'periods':duration,
        'before': get_UTC(before),
        'after': get_UTC(after),
        }
    res = requests.get(url,params=query).json()
    if ('error' in res):
        raise Exception(res['error'])

    Time,Open,High,Low,Close,Volume = [],[],[],[],[],[]
    for i in res['result'][str(duration)]:
        Time.append(datetime.fromtimestamp(i[0]).strftime('%Y-%m-%d'))
        Open.append(i[1])
        High.append(i[2])
        Low.append(i[3])
        Close.append(i[4])
        Volume.append(i[5])

    return pd.DataFrame(
        {'Date':Time, 'BTC':Close})

def get_UTC(dt):
    return int(datetime.strptime(dt, '%Y-%m-%d %H:%M %z').timestamp())

## 時系列全データ

In [ ]:
df1 = get_OHLC('2021-12-31 12:00 +0900', '2015-1-01 12:00 +0900')
df1 = df1.set_index('Date')
df1 = df1[(df1[['BTC']] != 0).all(axis=1)]
df1.to_csv(csv_path1)
df1

,BTC
Date,
2015-06-26,32000
2015-06-27,31700
2015-06-28,31000
2015-06-29,30870
2015-06-30,31550
...,...
2021-12-27,5808996
2021-12-28,5820883
2021-12-29,5456831


# trainデータ構築

In [ ]:
df2 = get_OHLC('2017-12-31 12:00 +0900', '2015-1-01 12:00 +0900')
df2 = df2.set_index('Date')
df2 = df2[(df2[['BTC']] != 0).all(axis=1)]
df2.to_csv(csv_path2)
df2

,BTC
Date,
2015-06-26,32000
2015-06-27,31700
2015-06-28,31000
2015-06-29,30870
2015-06-30,31550
...,...
2017-12-27,1829900
2017-12-28,1793400
2017-12-29,1696730


# testデータ構築

In [ ]:
df3 = get_OHLC('2021-12-31 12:00 +0900', '2017-1-01 12:00 +0900')
df3 = df3.set_index('Date')
df3 = df3[(df3[['BTC']] != 0).all(axis=1)]
df3.to_csv(csv_path3)
df3

,BTC
Date,
2017-01-02,120750
2017-01-03,122408
2017-01-04,123138
2017-01-05,143743
2017-01-06,118936
...,...
2021-12-27,5808996
2021-12-28,5820883
2021-12-29,5456831
